In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

In [2]:
# Hyperparameters
batch_size = 64
image_size = 64
nz = 256 # Size of the latent vector (noise input to the generator)
ngf = 550 # Size of feature maps in the generator
ndf = 50 # Size of feature maps in the discriminator
num_epochs = 25
lr = 0.0002
beta1 = 0.5 # Beta1 hyperparam for Adam optimizers
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
print(device)

cpu


In [4]:
transform = transforms.Compose([
transforms.Resize(image_size),
transforms.CenterCrop(image_size),
transforms.ToTensor(),
transforms.Normalize([0.5], [0.5], [0.5]), # Normalize the images between -1 and 1
])

In [ ]:
dataset = dsets.ImageFolder(root=r"/kaggle/input/skyview-an-aerial-landscape-dataset/Aeri
al_Landscapes", #Update the path here
transform=transform)
os.makedirs(r'/kaggle/working/op')
save_folder =(r'/kaggle/working/op')

In [ ]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
class Generator(nn.Module):
  def __init__(self):
    super(Generator, self).__init__()
self.main = nn.Sequential(
nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
nn.BatchNorm2d(ngf * 8),
nn.ReLU(True),
nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
nn.BatchNorm2d(ngf * 4),
                           nn.ReLU(True),
nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
nn.BatchNorm2d(ngf * 2),
nn.ReLU(True),
nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
nn.BatchNorm2d(ngf),
nn.ReLU(True),
nn.ConvTranspose2d(ngf, 3, 4, 2, 1, bias=False),
nn.Tanh()
)
def forward(self, input):
  return self.main(input)

In [ ]:
# Discriminator model
class Discriminator(nn.Module):
def __init__(self):
super(Discriminator, self).__init__()
self.main = nn.Sequential(
nn.Conv2d(3, ndf, 4, 2, 1, bias=False),
nn.LeakyReLU(0.2, inplace=True),
nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
nn.BatchNorm2d(ndf * 2),
nn.LeakyReLU(0.2, inplace=True),
nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
nn.BatchNorm2d(ndf * 4),
nn.LeakyReLU(0.2, inplace=True),
nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
nn.BatchNorm2d(ndf * 8),
nn.LeakyReLU(0.2, inplace=True),
nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
nn.Sigmoid()
)
def forward(self, input):
  return self.main(input)

In [ ]:
# Create the generator and discriminator
netG = Generator().to(device)
netD = Discriminator().to(device)
# Loss function and optimizers
criterion = nn.BCELoss()
fixed_noise = torch.randn(64, nz, 1, 1, device=device)
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
for epoch in range(num_epochs):
with tqdm(total=len(dataloader), desc=f"Epoch [{epoch+1}/{num_epochs}]", unit="batch
", colour="green", smoothing=0.1) as pbar:
for i, data in enumerate(dataloader, 0):
# Update Discriminator network
netD.zero_grad()
real_images = data[0].to(device)
b_size = real_images.size(0)
label = torch.full((b_size,), 1., device=device)
output = netD(real_images).view(-1)
errD_real = criterion(output, label)
errD_real.backward()
# Generate fake images
noise = torch.randn(b_size, nz, 1, 1, device=device)
fake_images = netG(noise)
label.fill_(0.)
output = netD(fake_images.detach()).view(-1)
errD_fake = criterion(output, label)
errD_fake.backward()
optimizerD.step()
# Update Generator network
netG.zero_grad()
label.fill_(1.)
output = netD(fake_images).view(-1)
errG = criterion(output, label)
errG.backward()
optimizerG.step()
# Update progress bar
pbar.set_postfix({
"Loss_D": f"{(errD_real + errD_fake).item():.4f}",
"Loss_G": f"{errG.item():.4f}"
})
pbar.update(1) # Update the progress bar for each batch
# Save fake images after every epoch
with torch.no_grad():
fake_images = netG(fixed_noise).detach().cpu()
vutils.save_image(fake_images,os.path.join(save_folder, f"fake_images_epoch_{epoch}.png"), normalize=True)


In [ ]:
real_batch = next(iter(dataloader))
plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, norma
lize=True).cpu(), (1, 2, 0)))
plt.subplot(1, 2, 2)
plt.axis("off")
plt.title("Generated Images")
plt.imshow(np.transpose(vutils.make_grid(fake_images, padding=2, normalize=True).cpu(),
(1, 2, 0)))
plt.show()